In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torch import nn,optim
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_data=pd.read_csv(r'F:\study\ml\DataSet\california-house-prices\train.csv')
test_data=pd.read_csv(r'F:\study\ml\DataSet\california-house-prices\test.csv')

In [3]:
train_data.shape

(47439, 41)

In [4]:
test_data.shape

(31626, 40)

In [5]:
train_data.head()

,Id,Address,Sold Price,Summary,Type,Year built,Heating,Cooling,Parking,Lot,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,0,540 Pine Ln,3825000.0,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,1969.0,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",1.0,...,"Garage, Garage - Attached, Covered",886486.0,12580.0,2019-10-24,4198000.0,NaN,NaN,Los Altos,94022,CA
1,1,1727 W 67th St,505000.0,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926.0,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4047.0,...,"Detached Carport, Garage",505000.0,6253.0,2019-10-16,525000.0,2019-08-30,328000.0,Los Angeles,90047,CA
2,2,28093 Pine Ave,140000.0,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958.0,Forced air,NaN,0 spaces,9147.0,...,NaN,49627.0,468.0,2019-08-25,180000.0,NaN,NaN,Strawberry,95375,CA
3,3,10750 Braddock Dr,1775000.0,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947.0,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",NaN,...,"Detached Carport, Driveway, Garage - Two Door",1775000.0,20787.0,2019-10-24,1895000.0,2016-08-30,1500000.0,Culver City,90230,CA
4,4,7415 O Donovan Rd,1175000.0,Beautiful 200 acre ranch land with several pas...,VacantLand,NaN,NaN,NaN,0 spaces,NaN,...,NaN,NaN,NaN,2019-06-07,1595000.0,2016-06-27,900000.0,Creston,93432,CA


In [6]:
test_data.head()

,Id,Address,Summary,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,47439,3034 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-06,799900.0,2020-07-01,819000.0,Dodgertown,90090,CA
1,47440,565 Kenilworth Ave,duplex fixer. Input for comps only,SingleFamily,1924.0,Natural Gas,None,Detached,10018.8,3,...,Detached,521977.0,7494.0,2014-04-04,479950.0,2020-11-03,15000.0,San Leandro,94577,CA
2,47441,3028 N Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,NaN,NaN,2020-11-12,839900.0,NaN,NaN,Los Angeles,90039,CA
3,47442,3022 N North Coolidge Ave,Live within steps to the scenic views on the L...,SingleFamily,2020.0,Central,Central Air,Tandem Uncovered,940.0,2,...,Tandem Uncovered,442800.0,5370.0,2020-11-06,809900.0,2020-09-21,810000.0,Dodgertown,90090,CA
4,47443,2515 Admiral Cir,"This beautiful, spacious home built in 2017 is...",SingleFamily,2017.0,Forced Air,Central Air,"Garage Door Opener, Attached",2613.6,4,...,"Garage Door Opener, Attached",965282.0,12912.0,2020-12-02,1095000.0,2019-12-27,1041000.0,Hayward,94545,CA


In [7]:
train_label=train_data['Sold Price']

In [8]:
train_data.shape

(47439, 41)

In [9]:
train_data.drop(labels=['Sold Price'],axis=1,inplace=True)

In [10]:
train_data.shape

(47439, 40)

In [11]:
large_val_cols = ['Lot', 'Total interior livable area',
                  'Tax assessed value', 'Annual tax amount',
                  'Listed Price', 'Last Sold Price']

In [12]:
for c in large_val_cols:
    train_data[c]=np.log(train_data[c]+1)

In [13]:
for c in large_val_cols:
    test_data[c]=np.log(test_data[c]+1)

In [14]:
train_labels=np.log(train_label+1)

In [15]:
all_features=pd.concat((train_data.iloc[:,1:],test_data.iloc[:,1:]))

In [16]:
numeric_features=all_features.dtypes[all_features.dtypes != 'object'].index

In [17]:
numeric_features

Index(['Year built', 'Lot', 'Bathrooms', 'Full bathrooms',
       'Total interior livable area', 'Total spaces', 'Garage spaces',
       'Elementary School Score', 'Elementary School Distance',
       'Middle School Score', 'Middle School Distance', 'High School Score',
       'High School Distance', 'Tax assessed value', 'Annual tax amount',
       'Listed Price', 'Last Sold Price', 'Zip'],
      dtype='object')

In [18]:
numeric_features_name=numeric_features.tolist()
numeric_features_name

['Year built',
 'Lot',
 'Bathrooms',
 'Full bathrooms',
 'Total interior livable area',
 'Total spaces',
 'Garage spaces',
 'Elementary School Score',
 'Elementary School Distance',
 'Middle School Score',
 'Middle School Distance',
 'High School Score',
 'High School Distance',
 'Tax assessed value',
 'Annual tax amount',
 'Listed Price',
 'Last Sold Price',
 'Zip']

In [19]:
all_features[numeric_features].head(20)

,Year built,Lot,Bathrooms,Full bathrooms,Total interior livable area,Total spaces,Garage spaces,Elementary School Score,Elementary School Distance,Middle School Score,Middle School Distance,High School Score,High School Distance,Tax assessed value,Annual tax amount,Listed Price,Last Sold Price,Zip
0,1969.0,0.693147,0.0,NaN,0.693147,0.0,0.0,7.0,0.4,NaN,NaN,8.0,1.3,13.695022,9.439943,15.250119,NaN,94022
1,1926.0,8.305978,2.0,2.0,6.771936,1.0,1.0,3.0,0.8,2.0,1.1,2.0,1.3,13.132316,8.740977,13.171155,12.700772,90047
2,1958.0,9.121291,3.0,1.0,7.050123,0.0,0.0,NaN,NaN,NaN,NaN,NaN,10.1,10.812310,6.150603,12.100718,NaN,95375
3,1947.0,NaN,3.0,3.0,7.868254,0.0,0.0,9.0,0.2,7.0,0.2,8.0,0.2,14.389312,9.942131,14.454730,14.220976,90230
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.5,5.0,10.2,6.0,10.6,NaN,NaN,14.282385,13.710151,93432
5,1905.0,8.182280,2.0,2.0,7.179308,0.0,0.0,NaN,NaN,NaN,NaN,2.0,3.3,12.245683,7.836765,12.323416,12.206078,95202
6,1926.0,NaN,NaN,NaN,NaN,0.0,0.0,8.0,0.3,NaN,NaN,NaN,NaN,14.278616,9.863759,14.284890,13.122365,90039
7,2005.0,14.387140,2.0,2.0,7.832014,4.0,4.0,4.0,6.3,NaN,NaN,2.0,6.3,NaN,NaN,13.120363,NaN,95983
8,2001.0,NaN,3.0,3.0,7.379008,1.0,1.0,8.0,0.3,8.0,0.3,7.0,0.5,13.899014,9.531989,14.316286,NaN,94121
9,1973.0,11.100620,2.0,2.0,7.661056,0.0,0.0,6.0,0.6,NaN,NaN,7.0,1.5,12.030337,7.554859,13.864302,NaN,95003


In [20]:
numeric_features_name.remove('Zip')

In [21]:
numeric_features_name

['Year built',
 'Lot',
 'Bathrooms',
 'Full bathrooms',
 'Total interior livable area',
 'Total spaces',
 'Garage spaces',
 'Elementary School Score',
 'Elementary School Distance',
 'Middle School Score',
 'Middle School Distance',
 'High School Score',
 'High School Distance',
 'Tax assessed value',
 'Annual tax amount',
 'Listed Price',
 'Last Sold Price']

In [22]:
all_features[numeric_features_name].isna().sum()

Year built                      1942
Lot                            22989
Bathrooms                       5410
Full bathrooms                 12928
Total interior livable area     3878
Total spaces                    1667
Garage spaces                   1667
Elementary School Score         8735
Elementary School Distance      8493
Middle School Score            28279
Middle School Distance         28277
High School Score               7784
High School Distance            7175
Tax assessed value              6323
Annual tax amount               7209
Listed Price                       0
Last Sold Price                29545
dtype: int64

In [23]:
all_features[numeric_features_name]=all_features[numeric_features_name].apply(lambda x : (x-x.mean()) / x.std())

In [24]:
all_features[numeric_features_name].mean()

Year built                    -1.430044e-16
Lot                           -2.763149e-14
Bathrooms                     -1.436180e-14
Full bathrooms                 2.410896e-15
Total interior livable area    3.150520e-13
Total spaces                   1.849120e-14
Garage spaces                 -5.168904e-15
Elementary School Score        1.841204e-14
Elementary School Distance     2.700541e-16
Middle School Score           -1.330250e-15
Middle School Distance        -1.630889e-15
High School Score             -6.248362e-15
High School Distance           1.493850e-14
Tax assessed value            -4.857645e-14
Annual tax amount             -9.769432e-14
Listed Price                  -1.570527e-13
Last Sold Price               -1.278824e-13
dtype: float64

In [25]:
all_features[numeric_features_name]=all_features[numeric_features_name].fillna(all_features[numeric_features_name].mean())

In [26]:
n_train=train_data.shape[0]

In [27]:
all_features.head()

,Address,Summary,Type,Year built,Heating,Cooling,Parking,Lot,Bedrooms,Bathrooms,...,Parking features,Tax assessed value,Annual tax amount,Listed On,Listed Price,Last Sold On,Last Sold Price,City,Zip,State
0,540 Pine Ln,"540 Pine Ln, Los Altos, CA 94022 is a single f...",SingleFamily,4.850534e-02,"Heating - 2+ Zones, Central Forced Air - Gas","Multi-Zone, Central AC, Whole House / Attic Fan","Garage, Garage - Attached, Covered",-5.378320e+00,"Ground Floor Bedroom, Master Bedroom on Ground...",-2.058002e+00,...,"Garage, Garage - Attached, Covered",7.318137e-01,8.558527e-01,2019-10-24,1.759273,NaN,-1.278824e-13,Los Altos,94022,CA
1,1727 W 67th St,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,-2.846555e-01,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",-4.461969e-01,3,-3.367686e-01,...,"Detached Carport, Garage",1.853176e-01,1.278330e-01,2019-10-16,-0.421097,2019-08-30,-3.493563e-01,Los Angeles,90047,CA
2,28093 Pine Ave,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,-3.672186e-02,Forced air,NaN,0 spaces,8.201942e-02,2,5.238479e-01,...,NaN,-2.067855e+00,-2.570212e+00,2019-08-25,-1.543748,NaN,-1.278824e-13,Strawberry,95375,CA
3,10750 Braddock Dr,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,-1.219491e-01,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",-2.763149e-14,5,5.238479e-01,...,"Detached Carport, Driveway, Garage - Two Door",1.406103e+00,1.378915e+00,2019-10-24,0.925087,2016-08-30,1.219168e+00,Culver City,90230,CA
4,7415 O Donovan Rd,Beautiful 200 acre ranch land with several pas...,VacantLand,-1.430044e-16,NaN,NaN,0 spaces,-2.763149e-14,NaN,-1.436180e-14,...,NaN,-4.857645e-14,-9.769432e-14,2019-06-07,0.744336,2016-06-27,6.921063e-01,Creston,93432,CA


In [28]:
all_features[numeric_features_name].head()

,Year built,Lot,Bathrooms,Full bathrooms,Total interior livable area,Total spaces,Garage spaces,Elementary School Score,Elementary School Distance,Middle School Score,Middle School Distance,High School Score,High School Distance,Tax assessed value,Annual tax amount,Listed Price,Last Sold Price
0,4.850534e-02,-5.378320e+00,-2.058002e+00,2.410896e-15,-1.346885e+01,-1.355188e-01,-1.236235e-01,6.227580e-01,-3.392817e-01,-1.330250e-15,-1.630889e-15,9.293295e-01,-0.318177,7.318137e-01,8.558527e-01,1.759273,-1.278824e-13
1,-2.846555e-01,-4.461969e-01,-3.367686e-01,-1.064242e-01,-1.260004e+00,-6.273141e-02,-5.050667e-02,-1.308481e+00,-1.561423e-01,-1.607235e+00,-2.630715e-01,-2.101186e+00,-0.318177,1.853176e-01,1.278330e-01,-0.421097,-3.493563e-01
2,-3.672186e-02,8.201942e-02,5.238479e-01,-1.167570e+00,-7.012844e-01,-1.355188e-01,-1.236235e-01,1.841204e-14,2.700541e-16,-1.330250e-15,-1.630889e-15,-6.248362e-15,2.350838,-2.067855e+00,-2.570212e+00,-1.543748,-1.278824e-13
3,-1.219491e-01,-2.763149e-14,5.238479e-01,9.547215e-01,9.418787e-01,-1.355188e-01,-1.236235e-01,1.588378e+00,-4.308514e-01,8.654868e-01,-6.507716e-01,9.293295e-01,-0.651804,1.406103e+00,1.378915e+00,0.925087,1.219168e+00
4,-1.430044e-16,-2.763149e-14,-1.436180e-14,2.410896e-15,3.150520e-13,1.849120e-14,-5.168904e-15,1.399482e-01,3.369292e+00,-1.236020e-01,3.657007e+00,-8.084238e-02,2.502486,-4.857645e-14,-9.769432e-14,0.744336,6.921063e-01


In [29]:
train_features=torch.Tensor(all_features[numeric_features_name].iloc[:n_train,:].values)
test_features=torch.Tensor(all_features[numeric_features_name].iloc[n_train:,:].values)
train_labels=torch.Tensor(train_labels.values).reshape(-1,1)

In [30]:
loss=nn.MSELoss()

In [31]:
train_features.shape

torch.Size([47439, 17])

In [32]:
test_features.shape

torch.Size([31626, 17])

In [33]:
train_labels.shape

torch.Size([47439, 1])

In [34]:
def log_rmse(net,features,labels):
    clipped_preds=torch.clamp(net(features),1,float('inf'))
    rmse=torch.sqrt(loss(torch.log(clipped_preds),torch.log(labels)))
    return rmse.item()

In [35]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=10000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=log_rmse(net,test_features,test_labels)
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=log_rmse(net,train_features,train_labels)
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])
    print('train log loss: ',train_loss)
    print('test log loss: ',test_loss)
    return train_l,test_l

In [36]:
def get_kfold_data(k,j,x,y,random_state=13):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    row_list=list(range(x.shape[0]))
    np.random.seed(random_state)
    np.random.shuffle(row_list)
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[row_list[idx],:],y[row_list[idx],:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [37]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [38]:
def get_net():
    return nn.Sequential(nn.Linear(train_features.shape[1],256),nn.ReLU(),nn.Linear(256,128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,32),
                        nn.ReLU(),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,100000,0.00002,5,train_features,train_labels,256,0,0)

epoch = 1 train_loss : 2.618908 , test loss : 2.619977
epoch = 2 train_loss : 2.593651 , test loss : 2.592093
epoch = 3 train_loss : 1.925870 , test loss : 1.919929
epoch = 4 train_loss : 1.088391 , test loss : 1.083814
epoch = 5 train_loss : 0.616019 , test loss : 0.614149
epoch = 6 train_loss : 0.452931 , test loss : 0.453280
epoch = 7 train_loss : 0.403658 , test loss : 0.404644
epoch = 8 train_loss : 0.370481 , test loss : 0.371875
epoch = 9 train_loss : 0.347567 , test loss : 0.349157
epoch = 10 train_loss : 0.334459 , test loss : 0.335951
epoch = 11 train_loss : 0.318502 , test loss : 0.320050
epoch = 12 train_loss : 0.301175 , test loss : 0.302878
epoch = 13 train_loss : 0.292357 , test loss : 0.293897
epoch = 14 train_loss : 0.275675 , test loss : 0.277342
epoch = 15 train_loss : 0.263699 , test loss : 0.265316
epoch = 16 train_loss : 0.248922 , test loss : 0.250614
epoch = 17 train_loss : 0.235660 , test loss : 0.237372
epoch = 18 train_loss : 0.224543 , test loss : 0.226205
e

epoch = 172 train_loss : 0.018129 , test loss : 0.019579
epoch = 173 train_loss : 0.018116 , test loss : 0.019563
epoch = 174 train_loss : 0.018044 , test loss : 0.019467
epoch = 179 train_loss : 0.017990 , test loss : 0.019397
epoch = 180 train_loss : 0.017927 , test loss : 0.019364
epoch = 181 train_loss : 0.017904 , test loss : 0.019324
epoch = 185 train_loss : 0.017816 , test loss : 0.019251
epoch = 186 train_loss : 0.017792 , test loss : 0.019243
epoch = 187 train_loss : 0.017812 , test loss : 0.019221
epoch = 188 train_loss : 0.017778 , test loss : 0.019218
epoch = 189 train_loss : 0.017796 , test loss : 0.019199
epoch = 190 train_loss : 0.017709 , test loss : 0.019168
epoch = 193 train_loss : 0.017674 , test loss : 0.019126
epoch = 195 train_loss : 0.017623 , test loss : 0.019090
epoch = 197 train_loss : 0.017611 , test loss : 0.019073
epoch = 198 train_loss : 0.017582 , test loss : 0.019043
epoch = 200 train_loss : 0.017544 , test loss : 0.019023
epoch = 201 train_loss : 0.0175

epoch = 707 train_loss : 0.014861 , test loss : 0.016889
epoch = 714 train_loss : 0.014864 , test loss : 0.016886
epoch = 724 train_loss : 0.014825 , test loss : 0.016878
epoch = 728 train_loss : 0.014829 , test loss : 0.016873
epoch = 731 train_loss : 0.014811 , test loss : 0.016861
epoch = 751 train_loss : 0.014785 , test loss : 0.016856
epoch = 772 train_loss : 0.014772 , test loss : 0.016855
epoch = 779 train_loss : 0.014730 , test loss : 0.016850
epoch = 783 train_loss : 0.014740 , test loss : 0.016834
epoch = 800 train_loss : 0.014696 , test loss : 0.016815
epoch = 822 train_loss : 0.014650 , test loss : 0.016815
epoch = 846 train_loss : 0.014623 , test loss : 0.016802
epoch = 864 train_loss : 0.014602 , test loss : 0.016790
epoch = 880 train_loss : 0.014572 , test loss : 0.016779
epoch = 911 train_loss : 0.014538 , test loss : 0.016770
epoch = 935 train_loss : 0.014506 , test loss : 0.016758
epoch = 938 train_loss : 0.014497 , test loss : 0.016755
epoch = 939 train_loss : 0.0145

KeyboardInterrupt: 

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(train_features.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU(),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,100000,0.0002,5,train_features,train_labels,128,0,0)